<a href="https://colab.research.google.com/github/Fuenfgeld/DMA2022TeamA/blob/main/ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Erstellen der Quelldatenbank 
import requests
exec(requests.get('https://raw.githubusercontent.com/Fuenfgeld/DMA2022TeamA/main/quelldatenbank.py').text)

Mounted at /content/gdrive
Tabellen erfolgreich erstellt
Daten erfolgreich eingefügt


In [ ]:
# Initialisierung der Datawarehouse-Tabellen:
Datawh_Tabellen = {} 


In [ ]:
#Erstellen der Tabellen mit Schlüsseln
Datawh_Tabellen['Zentrum'] = """
  create table Zentrum(
    PATIENT_ID VARCHAR REFERENCES dimPATIENT(ID),
    ENCOUNTER_ID VARCHAR REFERENCES dimENCOUNTERS(ID),
    OBSERVATION_ID INTEGER REFERENCES dimOBSERVATIONS(ID),
    PROCEDURE_DATE VARCHAR REFERENCES dimPROCEDURE(DATE),
    PROCEDURE_CODE VARCHAR REFERENCES dimPROCEDURE(CODE),
    CONDITIONS_CODE VARCHAR REFERENCES dimCONDITIONS(CODE)
  );
"""

Datawh_Tabellen['dimObservations'] = """
  create table if not exists dimObservations(
    ID VARCHAR PRIMARY KEY,
    CODE VARCHAR,
    DESCRIPTION VARCHAR,
    VALUE VARCHAR,
    UNITS VARCHAR
  );
"""

Datawh_Tabellen['dimEncounters'] = """
  create table if not exists dimEncounters(
    ID VARCHAR PRIMARY KEY,
    START TIMESTAMP,
    STOP TIMESTAMP,
    ENCOUNTERCLASS VARCHAR,
    CODE VARCHAR,
    DESCRIPTION VARCHAR
  );
"""

Datawh_Tabellen['dimProcedures'] = """
  create table if not exists dimProcedures(
    DATE TIMESTAMP,
    PATIENT_ID VARCHAR,
    ENCOUNTER_ID VARCHAR,
    CODE VARCHAR,
    DESCRIPTION VARCHAR,
    PRIMARY KEY (DATE, PATIENT_ID, ENCOUNTER_ID, CODE)
  );
"""

Datawh_Tabellen['dimConditions'] = """
  create table if not exists dimConditions(
    PATIENT_ID VARCHAR,
    ENCOUNTER_ID,
    CODE VARCHAR,
    DESCRIPTION VARCHAR,
    PRIMARY KEY (PATIENT_ID, ENCOUNTER_ID, CODE)
  );
"""

Datawh_Tabellen['dimPatients'] = """
  create table if not exists dimPatients(
    ID VARCHAR PRIMARY KEY,
    DATASET_ORIGIN VARCHAR
  );
"""





In [ ]:
#Schreiben der Tabellen in eine Datenbank 
import sqlite3
def connect_to_db(db_file):
    sqlite3_conn = None
    try:
        sqlite3_conn = sq.connect(db_file)
        return sqlite3_conn

    except Error as err:
        print(err)

        if sqlite3_conn is not None:
            sqlite3_conn.close()
  
conn_dwh = sqlite3.connect('Datawarehouse.db')
if conn_dwh is not None:
        cursor_dwh = conn_dwh.cursor()
        for name in Datawh_Tabellen.keys():
          print (name)
          cursor_dwh.execute(Datawh_Tabellen[name])

else:
        print('Connection to database failed')

Zentrum
dimObservations
dimEncounters
dimProcedures
dimConditions
dimPatients


In [ ]:
#Einfügen der Werte in Zentrum:
Zentrum = pd.read_sql_query("""
SELECT t4.patient_id,
       t4.encounter_id,
       t4.observation_id,
       t4.procedure_date,
       t4.procedure_code,
       t5.code AS CONDITIONS_CODE
FROM   (SELECT t2.patient_id,
               t2.encounter_id,
               t2.observation_id,
               t3.date AS PROCEDURE_DATE,
               t3.code AS PROCEDURE_CODE
        FROM   (SELECT encounters.patient_id AS PATIENT_ID,
                       encounters.id         AS ENCOUNTER_ID,
                       t1.id                 AS OBSERVATION_ID               
                FROM   encounters
                       LEFT JOIN (SELECT id,
                                         patient_id,
                                         encounter_id
                                  FROM   observations
                                  WHERE  observations.code IN (
                                         "8480-6", "8462-4", "9279-1",
                                         "8867-4",
                                         "3569", "8310-5", "1975-2",
                                         "1920-8",
                                         "1742-6", "6768-6", "33914-3",
                                         "2885-2"
                                         ,
                                         "3094-0", "94531-1" )) t1
                              ON encounters.patient_id = t1.patient_id
                                 AND encounters.id = t1.encounter_id) t2
               LEFT JOIN (SELECT date,
                                 code,
                                 patient_id,
                                 encounter_id
                          FROM   procedures) t3
                      ON t2.patient_id = t3.patient_id
                         AND t2.encounter_id = t3.encounter_id) t4
       LEFT JOIN (SELECT patient_id,
                         encounter_id,
                         code
                  FROM   conditions
                  WHERE  code IN ( "840544004", "840539006" )) t5
              ON t4.patient_id = t5.patient_id
                 AND t4.encounter_id = t5.encounter_id 
;""", conn
  )
Zentrum.to_sql(name = 'Zentrum', con=conn_dwh, if_exists='append', index=False)
Zentrum.head(3)


,patient_id,encounter_id,observation_id,PROCEDURE_DATE,PROCEDURE_CODE,CONDITIONS_CODE
0,3575b903-dbd0-1d55-6146-9e8aa4ed52a5,fd59905a-4ec8-62a6-d45d-d2a458d60111,NaN,None,None,None
1,3575b903-dbd0-1d55-6146-9e8aa4ed52a5,de8159bf-90ae-c9a6-dd8d-f1de7b7972a7,NaN,None,None,None
2,3575b903-dbd0-1d55-6146-9e8aa4ed52a5,6e0279b7-c477-a2c6-e75c-a911f03a264c,NaN,None,None,None


In [ ]:
#Einfügen der Werte in Observations
dimObservations = pd.read_sql_query("""
Select ID, CODE, DESCRIPTION, VALUE, UNITS from observations
where observations.code in ("8480-6","8462-4","9279-1","8867-4","3569","8310-5","1975-2","1920-8","1742-6","6768-6","33914-3","2885-2","3094-0","94531-1")
;""", conn
  )

dimObservations.to_sql(name = 'dimObservations', con=conn_dwh, if_exists='append', index=False)
dimObservations.head(3)


,Id,code,description,value,units
0,5,8462-4,Diastolic Blood Pressure,82.0,mm[Hg]
1,6,8480-6,Systolic Blood Pressure,110.0,mm[Hg]
2,7,8867-4,Heart rate,65.0,/min


In [ ]:
#Einfügen der Werte in Encounters
dimEncounters = pd.read_sql_query("""
Select ID, START, STOP, ENCOUNTERCLASS, CODE, DESCRIPTION from encounters
;""", conn
  )

dimEncounters.to_sql(name = 'dimEncounters', con=conn_dwh, if_exists='append', index=False)

dimEncounters.head(3)

,id,start,stop,encounterclass,code,description
0,fd59905a-4ec8-62a6-d45d-d2a458d60111,1943-02-15T11:40:19Z,1943-02-15T11:55:19Z,ambulatory,185347001,Encounter for problem
1,de8159bf-90ae-c9a6-dd8d-f1de7b7972a7,1943-02-26T11:40:19Z,1943-02-26T11:55:19Z,ambulatory,185347001,Encounter for problem
2,6e0279b7-c477-a2c6-e75c-a911f03a264c,1946-01-11T11:40:19Z,1946-01-11T11:55:19Z,ambulatory,185345009,Encounter for symptom


In [ ]:
#Einfügen der Werte in Procedures
dimProcedures = pd.read_sql_query("""
select DATE,PATIENT_ID, ENCOUNTER_ID, CODE, DESCRIPTION from procedures
;""", conn
  )

dimProcedures.to_sql(name = 'dimProcedures', con=conn_dwh, if_exists='append', index=False)

dimProcedures.head(3)

,date,patient_id,encounter_id,code,description
0,2012-12-12T14:56:19Z,3575b903-dbd0-1d55-6146-9e8aa4ed52a5,f222d3d6-d9f0-3586-70d4-69a77f16b58c,73761001,Colonoscopy
1,2015-05-04T11:40:19Z,3575b903-dbd0-1d55-6146-9e8aa4ed52a5,c57b93a2-0302-4965-9e2c-713b84e7c24b,430193006,Medication Reconciliation (procedure)
2,2017-12-11T15:25:19Z,3575b903-dbd0-1d55-6146-9e8aa4ed52a5,11306017-08e3-1375-fc44-1265a8114949,73761001,Colonoscopy


In [ ]:
#Einfügen der Werte in Conditions
dimConditions = pd.read_sql_query("""
select PATIENT_ID, CODE, DESCRIPTION, ENCOUNTER_ID from conditions
;""", conn
  )

dimConditions.to_sql(name = 'dimConditions', con=conn_dwh, if_exists='append', index=False)

dimConditions.head(3)

,patient_id,code,description,encounter_id
0,3575b903-dbd0-1d55-6146-9e8aa4ed52a5,232353008,Perennial allergic rhinitis with seasonal vari...,6e0279b7-c477-a2c6-e75c-a911f03a264c
1,3575b903-dbd0-1d55-6146-9e8aa4ed52a5,162864005,Body mass index 30+ - obesity (finding),40ca71e3-5668-dd3a-d7e1-209a43835384
2,17f0c6d9-8931-8839-66cb-3ca6fb066d3e,162864005,Body mass index 30+ - obesity (finding),dbaad3dc-c5a8-9daf-5d5f-330a8022263f


In [ ]:
#Einfügen der Werte in Patients
dimPatients = pd.read_sql_query("""
select ID, DATASET_ORIGIN FROM patients
;""", conn
  )

dimPatients.to_sql(name = 'dimPatients', con=conn_dwh, if_exists='append', index=False)

dimPatients.head(3)

,id,dataset_origin
0,0008c0a7-f8f5-5967-965f-aa9e9c8b5567,asthma
1,001c3804-81e8-740e-dd91-973b584a2dd9,lung_cancer
2,001ef298-1d71-119b-3489-b51105b5edc6,dermatitis
